In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
# @vla: custom parse options and quantize function
from lavis.args_parser import *
from lavis.quantize import quantize

import lavis.tasks as tasks
from lavis.common.config import Config

In [18]:
parser = args_parser()

In [21]:
CLI_INPUT = f'''
                 --cfg-path ret_flickr_eval.yaml
                 --visual-encoder-block-indices {' '.join(['0','1','2','3'])}
                 --visual-encoder-block-modules qkv proj fc1 fc2
                 --visual-encoder-block-weight-bits 8
                 --qformer-layer-indices {' '.join([str(i) for i in range(12)])}
                 --qformer-self-attention-modules query key value dense
                 --qformer-self-attention-weight-bits 8
                 --qformer-cross-attention-modules query key value dense
                 --qformer-cross-attention-weight-bits 6
                 --qformer-text-ff-modules intermediate output
                 --qformer-text-ff-weight-bits 8
                 --qformer-img-ff-modules intermediate_query output_query
                 --qformer-img-ff-weight-bits 4
                 
             '''

In [22]:
args = parser.parse_args(CLI_INPUT.split())
vars(args)

usage: ipykernel_launcher.py [-h] --cfg-path CFG_PATH
                             [--options OPTIONS [OPTIONS ...]]
                             [--visual-encoder-block-modules [{qkv,proj,fc1,fc2} ...]]
                             [--visual-encoder-block-indices [VISUAL_ENCODER_BLOCK_INDICES ...]]
                             [--visual-encoder-block-weight-bits {1,2,3,4,5,6,7,8}]
                             [--qformer-layer-indices [QFORMER_LAYER_INDICES ...]]
                             [--qformer-self-attention-modules [{query,key,value,dense} ...]]
                             [--qformer-self-attention-weight-bits {1,2,3,4,5,6,7,8}]
                             [--qformer-cross-attention-modules [{query,key,value,dense} ...]]
                             [--qformer-cross-attention-weight-bits {1,2,3,4,5,6,7,8}]
                             [--qformer-text-ff-modules [{intermediate,output} ...]]
                             [--qformer-text-ff-weight-bits {1,2,3,4,5,6,7,8}]
      

AttributeError: 'tuple' object has no attribute 'tb_frame'

In [6]:
cfg = Config(args)
cfg

In [7]:
task = tasks.setup_task(cfg)
task

In [8]:
model = task.build_model(cfg)
model

MODEL_NAME: eva_clip_g
Position interpolate from 16x16 to 26x26


Blip2Qformer(
  (visual_encoder): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14))
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (blocks): ModuleList(
      (0-38): 39 x Block(
        (norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=1408, out_features=4224, bias=False)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=1408, out_features=1408, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (drop_path): Identity()
        (norm2): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=1408, out_features=6144, bias=True)
          (act): GELU(approximate='none')
          (fc2): Linear(in_features=6144, out_features=1408, bias=True)
          (drop): Dropout(p=0.0, inplace=False)
        )
      )
    )


In [9]:
quantize(model, args)

In [11]:
model.Qformer.bert.encoder.layer

ModuleList(
  (0): BertLayer(
    (attention): BertAttention(
      (self): BertSelfAttention(
        (query): NBitLinearDynamic(in_features=768, out_features=768, bias=True | w=8, a=32)
        (key): NBitLinearDynamic(in_features=768, out_features=768, bias=True | w=8, a=32)
        (value): NBitLinearDynamic(in_features=768, out_features=768, bias=True | w=8, a=32)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (output): BertSelfOutput(
        (dense): NBitLinearDynamic(in_features=768, out_features=768, bias=True | w=8, a=32)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (crossattention): BertAttention(
      (self): BertSelfAttention(
        (query): NBitLinearDynamic(in_features=768, out_features=768, bias=True | w=8, a=32)
        (key): NBitLinearDynamic(in_features=1408, out_features=768, bias=True | w=8, a=32)
        (value): NBitLinearDynamic(in_features=1408, 

In [10]:
model

Blip2Qformer(
  (visual_encoder): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14))
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (blocks): ModuleList(
      (0-3): 4 x Block(
        (norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): NBitLinearDynamic(in_features=1408, out_features=4224, bias=False)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): NBitLinearDynamic(in_features=1408, out_features=1408, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (drop_path): Identity()
        (norm2): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=1408, out_features=6144, bias=True)
          (act): GELU(approximate='none')
          (fc2): Linear(in_features=6144, out_features=1408, bias=True)
          (drop): Dropout(p=0.0, inplace=False)
    